In [1]:
!pip install ortools
!pip install pulp
!pip install mip

     |████████████████████████████████| 13.7MB 236kB/s 
     |████████████████████████████████| 1.0MB 52.3MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


     |████████████████████████████████| 40.6MB 86kB/s 
     |████████████████████████████████| 48.0MB 90kB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from pulp import *
from mip import Model, xsum, maximize, BINARY
from scipy.optimize import linear_sum_assignment
from scipy.stats import lognorm
import itertools
import random
import operator
import matplotlib.pyplot as plt

In [3]:
start_date = '2020-01-20 08:30:00'
end_date = '2020-01-20 08:35:00'

df = pd.read_csv('/content/drive/MyDrive/ECE1724/yellow_tripdata_2020-01.csv')
df = df.drop(['VendorID', \
              'passenger_count', \
              'RatecodeID', \
              'store_and_fwd_flag', \
              'payment_type', \
              'fare_amount', \
              'extra', \
              'mta_tax', \
              'tip_amount', \
              'tolls_amount', \
              'improvement_surcharge',\
              'total_amount',\
              'congestion_surcharge',\
              'PULocationID',\
              'DOLocationID'] , axis=1)

df['tpep_pickup_datetime'] = pd.to_datetime(df.tpep_pickup_datetime)
df['tpep_dropoff_datetime'] = pd.to_datetime(df.tpep_dropoff_datetime)
df = df.sort_values(by=['tpep_pickup_datetime'])
mask = (df['tpep_pickup_datetime'] > start_date) & (df['tpep_pickup_datetime'] <= end_date)
dfs = df.loc[mask]
dfs['trip_duration'] = dfs.tpep_dropoff_datetime - dfs.tpep_pickup_datetime
dfs['trip_duration_second'] = dfs.apply(lambda row : row['trip_duration'].total_seconds(), axis = 1)

dfs.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


(450, 5)

In [4]:
dfs.shape

(450, 5)

In [5]:
df_instance = dfs.drop(["trip_duration", "tpep_pickup_datetime", "tpep_dropoff_datetime"], axis=1).reset_index(drop=True)

In [6]:
deficit = 1
estimated_sigma, loc, scale = lognorm.fit(df_instance['trip_distance'])
estimated_mu = np.log(scale)
l,w = df_instance.shape
drivers = []
n_cars = df_instance.shape[0] - deficit
for i in range(0, n_cars):
    column_name = 'car_' + str(i)
    drivers.append(column_name)
    dis_to_pickup = np.random.lognormal(estimated_mu, estimated_sigma, l)
    df_instance[column_name] = dis_to_pickup

In [7]:
df_instance.head()

,trip_distance,trip_duration_second,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,car_8,car_9,car_10,car_11,car_12,car_13,car_14,car_15,car_16,car_17,car_18,car_19,car_20,car_21,car_22,car_23,car_24,car_25,car_26,car_27,car_28,car_29,car_30,car_31,car_32,car_33,car_34,car_35,car_36,car_37,...,car_409,car_410,car_411,car_412,car_413,car_414,car_415,car_416,car_417,car_418,car_419,car_420,car_421,car_422,car_423,car_424,car_425,car_426,car_427,car_428,car_429,car_430,car_431,car_432,car_433,car_434,car_435,car_436,car_437,car_438,car_439,car_440,car_441,car_442,car_443,car_444,car_445,car_446,car_447,car_448
0,0.50,140.0,28.621835,0.702052,1.226757,0.677333,2.168134,14.355656,2.126091,1.718957,1.861089,4.651816,0.736960,0.990618,3.675817,0.467716,1.545756,3.871079,3.193015,2.697886,8.479196,1.131545,2.566944,19.563752,1.771995,2.556367,6.406084,6.145717,0.832373,0.731570,0.285399,1.776356,1.205215,1.001455,0.725924,5.974125,6.433445,0.131635,2.252668,4.018523,...,2.765127,0.683242,1.011829,6.277016,0.676402,1.591377,1.119387,2.821292,1.024602,1.894682,0.286989,14.038301,1.654368,2.796190,2.385951,5.031106,0.667554,1.892402,3.529680,2.225792,2.310852,1.065788,2.008465,2.507275,3.809548,1.151104,6.918540,0.777074,3.441782,2.851617,1.000389,1.482057,3.625447,2.136044,3.962370,1.341989,0.372720,0.917172,0.771172,2.602405
1,2.58,85115.0,0.730489,1.438904,1.069513,1.052253,0.391298,4.510647,3.235391,1.758880,0.466214,4.830605,0.339413,6.428329,5.448040,3.628003,1.229329,1.689606,1.477840,10.463332,8.434077,2.299166,0.964387,1.723557,35.475700,1.930228,0.311488,0.852746,4.376887,5.490129,3.888386,2.199008,1.663385,0.815032,6.685904,0.330671,3.473918,3.751776,2.001077,2.434943,...,9.364481,1.164069,1.622433,1.317311,7.341030,14.867987,2.373125,1.874127,1.549715,0.866798,3.622558,0.813936,0.155893,4.882724,2.817283,1.213784,4.772604,2.034339,2.568385,2.261236,1.036066,0.629165,3.854677,1.145620,0.905401,4.712347,7.685501,4.371960,1.615279,0.448176,0.853707,2.734495,2.102373,1.991909,1.680257,1.170758,1.867956,0.619460,0.430881,0.400777
2,1.41,460.0,1.418437,1.733942,0.655007,25.961922,1.716587,7.457814,4.678831,3.132508,1.186868,1.431190,1.157697,1.092660,1.075607,3.629517,2.135752,1.191652,1.462806,0.960395,12.941667,2.302233,0.681754,4.396354,2.028253,3.135317,0.605504,6.209734,2.330609,2.830177,0.679955,2.606320,0.967182,2.368087,1.919060,6.936785,6.543620,2.424462,8.711529,4.412963,...,1.023028,5.279105,1.163589,0.416233,0.557602,0.705188,3.148620,1.588102,1.809193,2.881000,0.708956,2.061352,2.702394,1.936581,0.338099,0.639289,2.177188,11.157826,7.365332,1.467430,5.176121,0.239240,0.463611,0.680588,2.567848,3.084330,0.368980,1.893806,4.185692,2.363825,1.468074,0.772570,1.179319,2.335430,4.274704,6.379230,0.847238,1.874811,2.792700,4.620913
3,2.84,684.0,1.077044,0.482464,12.144225,2.092046,0.796515,1.384375,1.167915,2.000650,1.897824,2.582102,0.116051,0.419855,0.322528,2.342379,1.826015,1.538048,0.389397,1.044740,3.767322,2.242830,16.751498,1.513200,0.711815,7.025391,1.055044,0.422912,3.875815,0.573370,0.879174,9.609570,4.629224,2.692299,0.946829,9.214733,0.370332,3.504255,11.204747,4.239307,...,3.437824,11.447146,2.769298,0.829526,7.871976,0.951818,1.281164,1.038181,1.495281,3.115826,3.532649,1.256003,2.442124,13.462344,4.442053,2.024396,6.372567,6.730843,0.460576,2.249003,2.353787,10.248999,1.414788,0.859913,0.797573,0.500332,2.491189,0.326295,3.880142,0.692806,1.704498,0.349382,2.674731,0.923894,1.514734,1.988014,1.622235,17.728614,4.679011,1.296951
4,2.50,427.0,3.784515,4.066765,7.518101,0.782755,0.444611,12.166873,14.632623,3.896433,0.319410,5.745254,3.233533,0.671689,0.426181,0.925640,0.797655,1.284826,0.456378,11.412356,2.270384,1.495451,2.152421,1.214799,1.270484,3.815427,10.541023,1.615037,5.182776,1.398404,2.595508,1.837390,0.681350,3.250887,2.239838,0.449608,1.008059,3.670789,0.763909,11.019703,...,5.048573,2.507209,7.644678,4.483086,3.037883,4.475636,3.960994,0.179355,1.950751,3.064826,1.787888,2.141918,9.122514,2.073137

In [8]:
sam = ['trip_distance', 'trip_duration_second', 'car_0', 'car_1', 'car_2', 'car_3', 'car_4', 'car_5', 'car_6', 'car_7', 'car_8']#, 'car_9', 'car_10', 'car_11' ]
drivers = ['car_0', 'car_1', 'car_2', 'car_3', 'car_4', 'car_5', 'car_6', 'car_7', 'car_8']#, 'car_9', 'car_10', 'car_11']

In [9]:
df_instance = df_instance[sam]

In [10]:
df_instance = df_instance.head(10)

In [11]:
drivers

['car_0',
 'car_1',
 'car_2',
 'car_3',
 'car_4',
 'car_5',
 'car_6',
 'car_7',
 'car_8']

In [12]:
df_instance.head(15)

,trip_distance,trip_duration_second,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,car_8
0,0.50,140.0,28.621835,0.702052,1.226757,0.677333,2.168134,14.355656,2.126091,1.718957,1.861089
1,2.58,85115.0,0.730489,1.438904,1.069513,1.052253,0.391298,4.510647,3.235391,1.758880,0.466214
2,1.41,460.0,1.418437,1.733942,0.655007,25.961922,1.716587,7.457814,4.678831,3.132508,1.186868
3,2.84,684.0,1.077044,0.482464,12.144225,2.092046,0.796515,1.384375,1.167915,2.000650,1.897824
4,2.50,427.0,3.784515,4.066765,7.518101,0.782755,0.444611,12.166873,14.632623,3.896433,0.319410
5,0.53,153.0,2.751753,1.493953,1.476273,1.768272,6.325477,2.195782,1.061591,2.192998,2.059906
6,1.06,348.0,1.482544,2.427592,2.934517,1.312555,1.587036,1.194512,0.226612,2.971030,2.481437
7,0.81,332.0,0.882384,8.198870,5.454736,1.387799,2.118297,0.422691,2.380214,2.804387,3.198810
8,6.27,572.0,1.652307,1.153580,6.302237,0.633290,4.057155,2.345557,0.397648,1.476065,0.381719
9,17.38,2150.0,1.787956,2.512631,0.479621,6.393755,4.112779,6.231513,0.882237,1.237500,1.963688


In [13]:
df_instance['price'] = 0.0
def price(trip):
  price = (0.3 * trip["trip_distance"] + 0.25 * (trip["trip_duration_second"]/60)) + 5
  return price

df_instance['price'] = df_instance.apply(lambda row: price(row), axis=1 )

In [14]:
df_instance.head()

,trip_distance,trip_duration_second,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,car_8,price
0,0.50,140.0,28.621835,0.702052,1.226757,0.677333,2.168134,14.355656,2.126091,1.718957,1.861089,5.733333
1,2.58,85115.0,0.730489,1.438904,1.069513,1.052253,0.391298,4.510647,3.235391,1.758880,0.466214,360.419833
2,1.41,460.0,1.418437,1.733942,0.655007,25.961922,1.716587,7.457814,4.678831,3.132508,1.186868,7.339667
3,2.84,684.0,1.077044,0.482464,12.144225,2.092046,0.796515,1.384375,1.167915,2.000650,1.897824,8.702000
4,2.50,427.0,3.784515,4.066765,7.518101,0.782755,0.444611,12.166873,14.632623,3.896433,0.319410,7.529167


In [15]:
def cost(trip, driver):
  trip_cost = (trip["trip_distance"] + trip[driver])*0.1
  cost = trip_cost - trip["price"] 
  return cost

In [16]:
for driver in drivers:
  df_instance[driver] = df_instance.apply(lambda row: cost(row, driver), axis=1 )

df_instance.drop(["price", "trip_distance", "trip_duration_second"], axis=1, inplace=True)
df_instance.reset_index(inplace=True, drop=True)

In [17]:
df_instance.head()

,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,car_8
0,-2.821150,-5.613128,-5.560658,-5.615600,-5.466520,-4.247768,-5.470724,-5.511438,-5.497224
1,-360.088784,-360.017943,-360.054882,-360.056608,-360.122703,-359.710769,-359.838294,-359.985945,-360.115212
2,-7.056823,-7.025272,-7.133166,-4.602474,-7.027008,-6.452885,-6.730784,-6.885416,-7.079980
3,-8.310296,-8.369754,-7.203577,-8.208795,-8.338348,-8.279563,-8.301209,-8.217935,-8.228218
4,-6.900715,-6.872490,-6.527357,-7.200891,-7.234706,-6.062479,-5.815904,-6.889523,-7.247226


In [18]:
df_instance.shape # (trips, cars)

(10, 9)

In [21]:
costs = df_instance.values
num_trips = len(costs)
num_cars = len(costs[0])

In [22]:
num_trips

10

In [23]:
num_cars

9

In [24]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [26]:
%%time
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_trips):
    for j in range(num_cars):
        x[i, j] = solver.IntVar(0, 1, '')

# Each worker is assigned to at most 1 task.
for i in range(num_trips):
    solver.Add(solver.Sum([x[i, j] for j in range(num_cars)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_cars):
    solver.Add(solver.Sum([x[i, j] for i in range(num_trips)]) == 1)

objective_terms = []
for i in range(num_trips):
    for j in range(num_cars):
        objective_terms.append(costs[i][j] * x[i, j])


solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()

CPU times: user 10.5 ms, sys: 971 µs, total: 11.4 ms
Wall time: 11.4 ms


In [27]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Total revenue = ', abs(solver.Objective().Value()), '\n')
    for i in range(num_trips):
        for j in range(num_cars):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('trip %d assigned to car %d.  revenue = %d' % (i, j, abs(costs[i][j])))

Total revenue =  427.49415738392156 

trip 0 assigned to car 3.  revenue = 5
trip 1 assigned to car 0.  revenue = 360
trip 2 assigned to car 2.  revenue = 7
trip 3 assigned to car 1.  revenue = 8
trip 4 assigned to car 4.  revenue = 7
trip 6 assigned to car 6.  revenue = 6
trip 7 assigned to car 5.  revenue = 6
trip 8 assigned to car 8.  revenue = 8
trip 9 assigned to car 7.  revenue = 17
